# Grid Search Results Analysis

## Function Fitting

In [ ]:
import pandas as pd
import os
from glob import glob

import warnings
from pandas.errors import ParserWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ParserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

results_dir = 'ff_results/'
all_files = glob(os.path.join(results_dir, '*.txt'))

In [ ]:
method_dict = {'lecun' : 'lecun_numer', 'glorot': 'glorot', 'std' : 'lecun_norm', 'power' : 'power', 'baseline' : 'baseline'}

# List to collect dataframes
dfs = []

for file_path in all_files:
    method = os.path.splitext(os.path.basename(file_path))[0]
    df = pd.read_csv(file_path, sep=', ')
    df['method'] = method_dict[method]
    
    # Ensure all expected columns are present
    for col in ['pow_res', 'pow_basis']:
        if col not in df.columns:
            df[col] = pd.NA
    
    dfs.append(df)

# Combine all into one DataFrame
gsdf = pd.concat(dfs, ignore_index=True)

gs = gsdf[['method', 'function', 'G', 'width', 'depth', 'pow_res', 'pow_basis', 'run', 'loss', 'l2']]

Save the data to a single csv for possible further processing.

In [ ]:
gs.to_csv(os.path.join(results_dir, 'grid_search.csv'), index=False)

In [ ]:
# Isolate the run with the median performance for confidence
gs_sorted = gs.sort_values("loss")

# Grouping columns, including pow_res and pow_basis
group_cols = ['method', 'function', 'G', 'width', 'depth', 'pow_res', 'pow_basis']

# Define a function to get the row with the median loss
def get_median_row(group):
    median_loss = group['loss'].median()
    # Use idxmin on absolute difference to median to break ties predictably
    idx = (group['loss'] - median_loss).abs().idxmin()
    return group.loc[[idx]]

# Apply the function group-wise and reset the index
mgs = gs_sorted.groupby(group_cols, dropna=False, group_keys=False).apply(get_median_row).reset_index(drop=True)

In [ ]:
# Filter to only 'power' method
power_df = mgs[mgs['method'] == 'power'].copy()

# Group by function and architecture (G, width, depth), and find row with minimal loss
best_power_configs = (
    power_df
    .groupby(['function', 'G', 'width', 'depth'], dropna=False, group_keys=False)
    .apply(lambda g: g.loc[g['loss'].idxmin()])
    .reset_index(drop=True)
)

# Drop pow_res and pow_basis from the whole filtered set
mgs_nopow = mgs.drop(columns=['pow_res', 'pow_basis', 'run'])

# Drop pow_res and pow_basis from best_power_configs too
best_power_configs_nopow = best_power_configs.drop(columns=['pow_res', 'pow_basis', 'run'])

# Filter out original 'power' rows from mgs_nopow
non_power_rows = mgs_nopow[mgs_nopow['method'] != 'power']

# Combine best 'power' rows with all other methods
fgs = pd.concat([non_power_rows, best_power_configs_nopow], ignore_index=True)

In [ ]:
fgs

At this point we have a dataframe called `fgs` with a single run per architecture, corresponding to the median results. For each function and each method, we proceed to calculate how many instances outperform the baseline in terms of:

a. the final loss:

In [ ]:
# Step 1: Extract baseline rows
baseline_df = fgs[fgs['method'] == 'baseline'][['function', 'G', 'depth', 'width', 'loss']]
baseline_df = baseline_df.rename(columns={'loss': 'baseline_loss'})

# Step 2: Filter the methods of interest
methods_of_interest = ['glorot', 'lecun_norm', 'lecun_numer', 'power']
fgs_comp = fgs[fgs['method'].isin(methods_of_interest)].copy()

# Step 3: Merge with baseline on matching config
merged = pd.merge(
    fgs_comp,
    baseline_df,
    on=['function', 'G', 'depth', 'width'],
    how='inner'
)

# Step 4: Compare losses
merged['beats_baseline'] = merged['loss'] < merged['baseline_loss']

# Step 5: Group and count
result = (
    merged.groupby(['function', 'method'])['beats_baseline']
    .sum()
    .reset_index(name='num_architectures')
)

num_base = baseline_df[baseline_df['function']=='f1'].shape[0]
result['percentage'] = 100*result['num_architectures']/num_base

In [ ]:
print(result)

b. the final $L^2$ error relative to the reference solution:

In [ ]:
# Step 1: Get baseline l2 values
baseline_l2 = fgs[fgs['method'] == 'baseline'][['function', 'G', 'depth', 'width', 'l2']]
baseline_l2 = baseline_l2.rename(columns={'l2': 'baseline_l2'})

# Step 2: Filter the methods of interest again if needed
fgs_comp_l2 = fgs[fgs['method'].isin(methods_of_interest)].copy()

# Step 3: Merge on config
merged_l2 = pd.merge(
    fgs_comp_l2,
    baseline_l2,
    on=['function', 'G', 'depth', 'width'],
    how='inner'
)

# Step 4: Compare l2 values
merged_l2['beats_baseline_l2'] = merged_l2['l2'] < merged_l2['baseline_l2']

# Step 5: Group and count
result_l2 = (
    merged_l2.groupby(['function', 'method'])['beats_baseline_l2']
    .sum()
    .reset_index(name='num_architectures')
)

result_l2['percentage'] = 100*result_l2['num_architectures']/num_base

In [ ]:
print(result_l2)

Finally, let's find the number of architectures that minimize the loss and the relative $L^2$ error at the same time:

In [ ]:
# Reuse the merged DataFrame that contains both loss and l2 comparisons
# First, make sure both baseline_loss and baseline_l2 are available

# Step 1: Merge baseline loss and l2 together
baseline_all = fgs[fgs['method'] == 'baseline'][['function', 'G', 'depth', 'width', 'loss', 'l2']]
baseline_all = baseline_all.rename(columns={'loss': 'baseline_loss', 'l2': 'baseline_l2'})

# Step 2: Merge with the methods of interest
fgs_comp_all = fgs[fgs['method'].isin(methods_of_interest)].copy()
merged_all = pd.merge(
    fgs_comp_all,
    baseline_all,
    on=['function', 'G', 'depth', 'width'],
    how='inner'
)

# Step 3: Compare both loss and l2
merged_all['beats_both'] = (
    (merged_all['loss'] < merged_all['baseline_loss']) &
    (merged_all['l2'] < merged_all['baseline_l2'])
)

# Step 4: Group and count
result_both = (
    merged_all.groupby(['function', 'method'])['beats_both']
    .sum()
    .reset_index(name='num_architectures')
)

result_both['percentage'] = 100*result_both['num_architectures']/num_base

In [ ]:
print(result_both)

## PDE Solving

In [ ]:
results_dir = 'pde_results/'
all_files = glob(os.path.join(results_dir, '*.txt'))

In [ ]:
method_dict = {'glorot': 'glorot', 'lecun' : 'lecun_numer', 'std' : 'lecun_norm', 'power' : 'power', 'baseline' : 'baseline'}

# List to collect dataframes
dfs = []

for file_path in all_files:
    method = os.path.splitext(os.path.basename(file_path))[0]
    df = pd.read_csv(file_path, sep=', ')
    df['method'] = method_dict[method]
    
    # Ensure all expected columns are present
    for col in ['pow_res', 'pow_basis']:
        if col not in df.columns:
            df[col] = pd.NA
    
    dfs.append(df)

# Combine all into one DataFrame
gsdf = pd.concat(dfs, ignore_index=True)

gs = gsdf[['method', 'pde', 'G', 'width', 'depth', 'pow_res', 'pow_basis', 'run', 'loss', 'l2']]

rename_dict = {'allen-cahn':'ac', 'burgers':'burgers', 'helmholtz':'helmholtz'}

gs['pde'] = gs['pde'].replace(rename_dict)

Save the data to a single csv for possible further processing.

In [ ]:
gs.to_csv(os.path.join(results_dir, 'grid_search.csv'), index=False)

In [ ]:
# Isolate the run with the median performance for confidence
gs_sorted = gs.sort_values("loss")

# Grouping columns, including pow_res and pow_basis
group_cols = ['method', 'pde', 'G', 'width', 'depth', 'pow_res', 'pow_basis']

# Define a function to get the row with the median loss
def get_median_row(group):
    s = group['loss'].dropna()
    if s.empty:
        return group.iloc[0:0]  # drop this experiment (no valid loss)
    med = s.median()
    idx = (s - med).abs().idxmin()
    return group.loc[[idx]]

# Apply the function group-wise and reset the index
mgs = gs_sorted.groupby(group_cols, dropna=False, group_keys=False).apply(get_median_row).reset_index(drop=True)

In [ ]:
# Filter to only 'power' method
power_df = mgs[mgs['method'] == 'power'].copy()

# Group by function and architecture (G, width, depth), and find row with minimal loss
best_power_configs = (
    power_df
    .groupby(['pde', 'G', 'width', 'depth'], dropna=False, group_keys=False)
    .apply(lambda g: g.loc[g['loss'].idxmin()])
    .reset_index(drop=True)
)

# Drop pow_res and pow_basis from the whole filtered set
mgs_nopow = mgs.drop(columns=['pow_res', 'pow_basis', 'run'])

# Drop pow_res and pow_basis from best_power_configs too
best_power_configs_nopow = best_power_configs.drop(columns=['pow_res', 'pow_basis', 'run'])

# Filter out original 'power' rows from mgs_nopow
non_power_rows = mgs_nopow[mgs_nopow['method'] != 'power']

# Combine best 'power' rows with all other methods
fgs = pd.concat([non_power_rows, best_power_configs_nopow], ignore_index=True)

At this point we have a dataframe called `fgs` with a single run per architecture, corresponding to the median results. For each pde and each method, we proceed to calculate how many instances outperform the baseline in terms of:

a. the final loss:

In [ ]:
# Step 1: Extract baseline rows
baseline_df = fgs[fgs['method'] == 'baseline'][['pde', 'G', 'depth', 'width', 'loss']]
baseline_df = baseline_df.rename(columns={'loss': 'baseline_loss'})

# Step 2: Filter the methods of interest
methods_of_interest = ['lecun_norm', 'lecun_numer', 'glorot', 'power']
fgs_comp = fgs[fgs['method'].isin(methods_of_interest)].copy()

# Step 3: Merge with baseline on matching config
merged = pd.merge(
    fgs_comp,
    baseline_df,
    on=['pde', 'G', 'depth', 'width'],
    how='inner'
)

# Step 4: Compare losses
merged['beats_baseline'] = merged['loss'] < merged['baseline_loss']

# Step 5: Group and count
result = (
    merged.groupby(['pde', 'method'])['beats_baseline']
    .sum()
    .reset_index(name='num_architectures')
)

num_base = baseline_df[baseline_df['pde']=='ac'].shape[0]
result['percentage'] = 100*result['num_architectures']/num_base

In [ ]:
print(result)

b. the final $L^2$ error relative to the reference solution:

In [ ]:
# Step 1: Get baseline l2 values
baseline_l2 = fgs[fgs['method'] == 'baseline'][['pde', 'G', 'depth', 'width', 'l2']]
baseline_l2 = baseline_l2.rename(columns={'l2': 'baseline_l2'})

# Step 2: Filter the methods of interest again if needed
fgs_comp_l2 = fgs[fgs['method'].isin(methods_of_interest)].copy()

# Step 3: Merge on config
merged_l2 = pd.merge(
    fgs_comp_l2,
    baseline_l2,
    on=['pde', 'G', 'depth', 'width'],
    how='inner'
)

# Step 4: Compare l2 values
merged_l2['beats_baseline_l2'] = merged_l2['l2'] < merged_l2['baseline_l2']

# Step 5: Group and count
result_l2 = (
    merged_l2.groupby(['pde', 'method'])['beats_baseline_l2']
    .sum()
    .reset_index(name='num_architectures')
)

result_l2['percentage'] = 100*result_l2['num_architectures']/num_base

In [ ]:
print(result_l2)

Finally, let's find the number of architectures that minimize the loss and the relative $L^2$ error at the same time:

In [ ]:
# Reuse the merged DataFrame that contains both loss and l2 comparisons
# First, make sure both baseline_loss and baseline_l2 are available

# Step 1: Merge baseline loss and l2 together
baseline_all = fgs[fgs['method'] == 'baseline'][['pde', 'G', 'depth', 'width', 'loss', 'l2']]
baseline_all = baseline_all.rename(columns={'loss': 'baseline_loss', 'l2': 'baseline_l2'})

# Step 2: Merge with the methods of interest
fgs_comp_all = fgs[fgs['method'].isin(methods_of_interest)].copy()
merged_all = pd.merge(
    fgs_comp_all,
    baseline_all,
    on=['pde', 'G', 'depth', 'width'],
    how='inner'
)

# Step 3: Compare both loss and l2
merged_all['beats_both'] = (
    (merged_all['loss'] < merged_all['baseline_loss']) &
    (merged_all['l2'] < merged_all['baseline_l2'])
)

# Step 4: Group and count
result_both = (
    merged_all.groupby(['pde', 'method'])['beats_both']
    .sum()
    .reset_index(name='num_architectures')
)

result_both['percentage'] = 100*result_both['num_architectures']/num_base

In [ ]:
print(result_both)